# Image-to-Image Translation with Conditional Adversarial Networks 翻訳

## リンク
- [論文](https://arxiv.org/pdf/1611.07004.pdf)
- [ConditionalGANの説明](https://qiita.com/triwave33/items/f6352a40bcfbfdea0476)

## Abstract

私たちは、画像対画像変換の問題に対する汎用的な解決策として、条件付き対立ネットワークを調べる。
これらのネットワークは、入力画像から出力画像へのマッピングを学習するだけでなく、
このマッピングを訓練するための損失関数を学習する。
これにより、伝統的に非常に異なる損失処方を必要とする問題に同じ一般的なアプローチを適用することが可能になる。
このアプローチは、ラベルマップから写真を合成し、エッジマップからオブジェクトを再構成し、
画像をカラー化するなど、他のタスクに効果的です。
実際、この論文に関連するpix2pixソフトウェアがリリースされて以来、
多くのインターネットユーザー（その多くはアーティスト）が私たちのシステムで独自の実験を行っており、
パラメータの調整を必要とせずに幅広い適用性と採用の容易さを実証しています。
コミュニティとして、私たちはマッピング関数を手作業ではなくなりました。
この作業は、私たちの損失関数を手作業で調整することなく合理的な結果を達成できることを示唆しています

## 1. Introduction

画像処理、コンピュータグラフィックス、およびコンピュータビジョンにおける多くの問題は、
入力画像を対応する出力画像に「翻訳」することによってもたらされ得る。 
概念が英語またはフランス語で表現されるように、シーンはRGB画像、グラディエントフィールド、
エッジマップ、セマンティックラベルマップなどとしてレンダリングされます。
自動言語変換と同様に、 十分な訓練データが与えられている場合に、シーンの可能な表現を別の表現に変換するタスクとして、
画像から画像への変換が含まれます（図1参照）。
伝統的に、これらの作業のそれぞれは、別個の専用機械（例えば、[15,24,19,8,10,52,32,38,17,57,61]）に取り組まれてきたが、 常に同じです。
ピクセルからピクセルを予測します。 
このホワイトペーパーの目標は、これらの問題の共通フレームワークを開発することです。

このコミュニティは、畳み込みニューラルネット（CNN）が多種多様な画像予測問題の背後にある共通の主題になりつつある現在、この方向で大きなステップを踏んでいます。
CNNは、結果の質を評価する目的である損失関数を最小限に抑えることを学びます。
学習プロセスは自動ですが、効果的な損失の設計には多くの手作業が必要です。
言い換えれば、私たちはまだCNNに最小限に抑えたいものを伝える必要があります。 
しかし、ミダス王のように、私たちが望むものを慎重にしなければなりません！ 素朴なアプローチをとって、予測された真のピクセルと真の真のピクセルとの間のユークリッド距離を最小にするようにCNNに依頼すると、ぼやけた結果を生む傾向がある[42,61]。 これは、すべてのもっともらしい出力を平均化することによってユークリッド距離が最小化され、ぼやけが生じるからです。 CNNに、実際に鮮明で現実的な画像を出力するなど、実際に欲しいことをさせるロス関数が浮上しているのは未解決の問題であり、一般的には専門知識が必要です。

「出力を現実と区別できないように」というような高レベルの目標だけを指定し、この目標を満たすのに適切な損失関数を自動的に学習することができれば、非常に望ましいことです。幸いにも、これはまさに最近提案されたGenerative Adversarial Networks（GANs）[23,12,43,51,62]によって行われていることです。 GANは、出力画像が真であるか偽であるかを分類しながら、この損失を最小限に抑えるために生成モデルを訓練する損失を知る。ぼやけた画像は明らかに偽装されているため、許容されません。 GANはデータに適応する損失を知るため、伝統的に非常に異なる種類の損失機能を必要とする多数のタスクに適用することができます。

本稿では、条件付き設定でGANを探索する。 GANがデータの生成モデルを学ぶのと同様に、条件付GAN（cGAN）は条件付き生成モデルを学習する[23]。これにより、cGANはイメージからイメージへの変換タスクに適しています。ここでは、入力イメージを条件にして対応する出力イメージを生成します。

GANは過去2年間に精力的に研究されており、本書で探究している技術の多くはこれまでに提案されています。 それにもかかわらず、これまでの論文は特定のアプリケーションに焦点を絞っており、イメージ・イメージ変換の汎用ソリューションとしてイメージ条件付きGANがどのように有効であるかは不明であった。 私たちの主な貢献は、さまざまな問題について、条件付きGANが合理的な結果を生むことを実証することです。 我々の第二の貢献は、良い結果を達成するのに十分な単純なフレームワークを提示し、いくつかの重要な建築上の選択の効果を分析することです。 コードは https://github.com/phillipi/pix2pix から入手できます。

## 2. Related work

### Structured losses for image modeling 

画像 - 画像変換の問題は、多くの場合、ピクセルごとの分類または回帰（例えば、[38,57,27,34,61]）として定式化される。 これらの形式は、各出力ピクセルが入力画像で与えられた他のすべてのピクセルと条件付きで独立していると見なす意味で、出力空間を「非構造化」として扱います。 条件付きGANは、代わりに構造化された損失を学習します。 構造化された損失は、出力の共同構成に不利益をもたらす。 条件付きランダムフィールド[9]、SSIMメトリック[55]、特徴マッチング[14]、ノンパラメトリックロス[36]、畳み込み擬似事前[56]などの文献では、 マッチング共分散統計に基づく損失[29]。 条件付きGANは、損失が学習される点で異なり、理論上、出力とターゲットの間で異なる可能性のある構造にペナルティを課すことができます。

### Conditional GANs

■条件付き設定でGANを最初に適用するわけではありません。以前の作品と並行した作品では、GANを個別ラベル[40,22,12]、テキスト[45]、実際には画像に調整しています。画像条件付きモデルは、ノーマルマップ[54]、未来フレーム予測[39]、プロダクト写真生成[58]、およびスパースアノテーションからの画像生成[30]、[47]同じ問題へのアプローチ）。他のいくつかの論文では、イメージとイメージのマッピングにGANも使用していましたが、GANを無条件に適用し、出力を入力条件に強制的に適用するための他の条件（L2回帰など）に依存していました。これらの論文は、インペインティング[42]、将来の状態予測[63]、ユーザの制約[64]、スタイル移送[37]、超解像[35]各メソッドは特定のアプリケーションに合わせて調整されています。私たちの枠組みは、アプリケーション特有のものがないという点で異なります。これにより、セットアップは他のほとんどの設定よりもかなり簡単になります。我々の方法はまた、ジェネレータとディスクリミネータのいくつかのアーキテクチャ上の選択肢において、従来の研究とは異なる。

過去の作業とは異なり、私たちは "U-Net"ベースのアーキテクチャ[49]を使用しており、我々の識別器には畳み込みの "PatchGAN"分類器を使用しており、画像パッチの規模でのみ構造をペナルティする。類似のPatchGANアーキテクチャは、地方のスタイル統計を捕捉する目的で[37]で以前に提案されました。ここでは、このアプローチがより広い範囲の問題に効果的であることを示し、パッチサイズの変更の影響を調べます。

## 3. Method

GANは、ランダム雑音ベクトルzから出力画像y、G：z→yへの写像を学習する生成モデルである[23]。 対照的に、条件付きGANは、観測された画像xおよびランダムノイズベクトルzから、y、G：{x、z}→yへのマッピングを学習する。 ジェネレータGは、ジェネレータの「フェイク」を検出する際に可能な限り行うように訓練された対抗的に訓練された弁別器Dによって「実際の」画像と区別できない出力を生成するように訓練される。 このトレーニング手順は図2に示されています。

### 3.1. Objective

条件付きGANの目的は、

LcGAN (G, D) =Ex,y[log D(x, y)]+Ex,z[log(1 − D(x, G(x, z))], (1)

ここで、Gは最大化しようとする敵対的D、すなわちG * = arg minG maxD LcGAN（G、D）に対してこの目的を最小化しようとする。 弁別器を調整することの重要性をテストするために、弁別器がxを観察しない無条件の変形と比較する：

LGAN (G, D) =Ey[log D(y)]+Ex,z[log(1 − D(G(x, z))]. (2)

これまでのアプローチでは、GANの目的をL2距離などのより伝統的な損失と混在させることが有益であることが分かりました[42]。 弁別器の仕事は変わらないが、発電機は弁別器をだますだけでなく、L2の意味で地面の真理値出力に近くなるように任されている。 また、L1ではL1よりもL1距離を使用しているので、ぼやけが少なくなります。

LL1(G) = Ex,y,z[ky − G(x, z)k1]. (3)

私たちの最終目標は

G∗ = arg minG maxD LcGAN (G, D) + λLL1(G). (4)

zがなければ、ネットはxからyへのマッピングをまだ知ることができますが、決定論的な出力を生成するため、デルタ関数以外の分布には一致しません。 過去の条件付きGANはこれを認め、xに加えてジェネレータへの入力としてガウス雑音zを提供した（例えば、[54]）。 最初の実験では、この戦略は効果的ではありませんでした - 発電機はノイズを無視することを単純に学びました。これはMathieu et al。 [39]。 代わりに、私たちの最終的なモデルでは、訓練とテストの両方の時間に、発電機のいくつかの層に適用されるドロップアウトの形でしかノイズを供給しません。 ドロップアウトノイズがあるにもかかわらず、私たちはネットの出力においてわずかな確率論しか観察しません。 非常に確率的な出力を生成し、それによってモデル化された条件付き分布の完全なエントロピーを捕捉する条件付きGANを設計することは、本研究によって残された重要な問題である。

### 3.2. Network architectures

私たちは、ジェネレータとディスクリミネータのアーキテクチャを[43]のものから適応させます。 ジェネレータとディスクリミネータは、convolution-BatchNorm-ReLu [28]という形式のモジュールを使用します。 アーキテクチャの詳細は、オンラインで補足資料に記載されており、以下で説明する重要な機能を備えています。

#### 3.2.1 Generator with skips

イメージからイメージへの変換の問題の特徴は、高解像度の入力グリッドを高解像度の出力グリッドにマッピングすることです。さらに、私たちが考慮する問題については、入力と出力は表面の外観が異なりますが、どちらも同じ基礎構造のレンダリングです。したがって、入力の構造は出力の構造とおおむね一致します。

我々は、これらの考慮事項に基づいてジェネレータアーキテクチャを設計する。この分野の問題に対する多くの従来の解決法[42,54,29,63,58]は、エンコーダ - デコーダネットワーク[25]を使用してきた。このようなネットワークでは、入力は一連のレイヤーを通過し、そのレイヤーはボトルネックレイヤーまで漸進的にダウンサンプリングされ、その時点でプロセスが逆になります。このようなネットワークでは、すべての情報フローがボトルネックを含むすべてのレイヤーを通過する必要があります。多くの画像変換の問題では、入力と出力の間で多量の低レベルの情報が共有されており、ネット上で直接この情報をシャトルすることが望ましいでしょう。例えば、画像色彩化の場合、入力および出力は顕著なエッジの位置を共有する。

このような情報のボトルネックを回避する手段を発生器に与えるために、「U-Net」[49]の一般的な形に従って、スキップ接続を追加します。具体的には、各レイヤーiとレイヤーn-iの間にスキップ接続を追加します。ここで、nはレイヤーの総数です。各スキップ接続は、レイヤiのすべてのチャネルをレイヤn-iのチャネルと単に連結するだけです。

#### 3.2.2 Markovian discriminator (PatchGAN)

L2損失 - とL1（図4参照） - 画像生成問題でぼやけた結果が生じることはよく知られている[33]。これらの損失は高周波の鮮明度を促進することはできませんが、多くの場合、低周波を正確に捕捉します。このような場合の問題については、低い周波数で正確さを実現する全く新しいフレームワークは必要ありません。 L1は既に行います。これは、低周波の正確さを強制するためにL1項に依存して、GAN弁別器を高周波構造のみをモデル化するように制限する動機となる（方程式4）。

高周波数をモデル化するには、ローカル画像パッチの構造に注意を払うだけで十分です。したがって、私たちはパッチのスケールでのみ構造にペナルティをかける識別器アーキテクチャ（PatchGANと呼ばれます）を設計します。この弁別子は、画像内の各N×Nパッチが真であるか偽であるかを分類しようと試みる。我々は、この識別器を画像全体にわたって畳み込み演算し、すべての応答を平均してDの最終出力を提供する.4.4節では、Nが画像のフルサイズよりもずっと小さく、高品質の結果を生成することを示す。

これは、より小さいPatchGANがより少ないパラメータしか持たず、より速く実行され、任意に大きな画像に適用できるので、有利である。このような弁別器は、パッチ直径よりも離れた画素間の独立性を仮定して、マルコフランダム場として画像を効果的にモデル化する。この接続は、以前に[37]で探求されており、テクスチャ[16,20]およびスタイル[15,24,21,36]のモデルにおける共通の仮定でもある。したがって、PatchGANはテクスチャ/スタイルの損失の一形態として理解することができます。

### 3.3. Optimization and inference

我々のネットワークを最適化するために、我々は[23]からの標準的なアプローチに従う：Dの勾配降下ステップの1つとGの1つのステップを交互に繰り返す。元のGAN論文で提案されているように、 （x、G（x、z））を最大化するように訓練する代わりに、Dを最適化しながら目的を2で除算して、速度を遅くする我々はminibatch SGDを使用し、学習率0.0002、運動量パラメータβ1= 0.5、β2= 0.999のAdamソルバ[31]を適用する。

推論時に、我々は全く同じジェネレータネットを実行するこれはテスト時にドロップアウトを適用し、トレーニングバッチの集計統計ではなく、テストバッチの統計を使用してバッチ正規化[28]を適用する点で通常のプロトコルとは異なります。バッチサイズが1に設定されている場合、バッチ標準化は「インスタンス正規化」と呼ばれ、画像生成タスク[53]において有効である。実験では、実験に応じて1〜10のバッチサイズを使用します。

## 4. Experiments

条件付きGANの一般性を調べるために、我々は写真の生成のようなグラフィックスタスクとセマンティックセグメンテーションのようなビジョンタスクの両方を含む、さまざまなタスクとデータセットのメソッドをテストします。

- セマンティックラベル↔写真は、Cityscapesデータセット[11]で訓練されています。 
- CMP Facades [44]で訓練された建築ラベル→写真。 
- Googleマップからスクレイプされたデータを訓練したMap緯度写真。 
- BW→[50]で訓練されたカラー写真。 
- [64]と[59]のデータを訓練したエッジ→写真。HEDエッジ検出器[57]と後処理を使用して生成されたバイナリエッジ。 
- スケッチ→写真：[18]からhumandrawnスケッチのエッジ→写真モデルをテストします。 
- 昼から夜、[32]で訓練を受けた。 
- [熱]→カラー写真、[26]のデータを訓練。 
- ピクセルが欠けている写真→[13]からParis StreetViewで訓練されたインペイントされた写真。

これらの各データセットのトレーニングの詳細は、補足資料オンラインで提供されています。 すべての場合、入力と出力は単なる1〜3チャネルの画像です。 定性的結果を図8、図9、図11、図10、図12、図13、図14、図15、図16、図17、図18、図19に示します。図20にいくつかの失敗例が示されています。https：// phillipi .github.io / pix2pix /。

- Data requirements and speed

小さなデータセットでもしばしば適切な結果が得られることに注意してください。 当社のファサードトレーニングセットはわずか400枚の画像で構成され（図13の結果を参照）、昼夜のトレーニングセットは91のユニークなウェブカメラで構成されています（図14の結果を参照）。 このサイズのデータセットでは、トレーニングが非常に高速になる可能性があります。たとえば、図13に示す結果は、単一のPascal Titan X GPUで2時間未満のトレーニングでした。 テスト時には、すべてのモデルがこのGPU上で1秒未満で動作します。

### 4.1. Evaluation metrics

合成された画像の品質を評価することは、オープンで困難な問題である[51]。ペルピクセル平均二乗誤差などの従来のメトリックでは、結果の共同統計は評価されないため、構造化された損失が捕捉しようとする構造を測定することはありません。結果の視覚的品質をより総合的に評価するために、私たちは2つの方法を採用しています。まず、Amazon Mechanical Turk（AMT）の「真実と偽の」知覚研究を実行します。カラー化や写真生成のようなグラフィックスの問題の場合、人間の観察者への妥当性は、しばしば究極の目標です。したがって、この手法を使用して、地図生成、空中写真生成、および画像の色付けをテストします。次に、合成された都市景観が現実的なものであるかどうかを、既製の認識システムが認識できるようにするかどうかを測定する。このメトリックは、[51]の「開始スコア」、[54]のオブジェクト検出評価、および[61]および[41]の「意味解釈能力」尺度に類似しています。

- AMT perceptual studies 

私たちのAMT実験では、我々は[61]からのプロトコルに従った：Turkersは、我々のアルゴリズムによって生成された "偽の"画像に対して "実際の"画像をピットした一連の試行を提示された。各試行では、各画像が1秒間現れ、その後画像は消え、Turkersは無制限の時間を与えられて偽のものとして応答した。各セッションの最初の10枚は練習であり、トゥルカーはフィードバックを受けました。主な実験の40回の試験ではフィードバックは得られなかった。各セッションで一度に1つのアルゴリズムしかテストされず、Turkersは複数のセッションを完了することができませんでした。 〜50人のトルコ人が各アルゴリズムを評価した。 [61]とは異なり、我々は警戒試験を含まなかった。私たちのカラー化実験では、実際の画像と偽の画像が同じグレースケール入力から生成されました。 map匿名の写真では、タスクをより難しくし、フロアレベルの結果を避けるために、同じ入力から実際のイメージと偽のイメージが生成されませんでした。マップ緯度写真については、256×256の解像度の画像を訓練したが、512×512の画像をテストするために完全畳み込み変換を利用した。この画像はダウンサンプリングされ、256×256の解像度でTurkersに提示された。カラー化のために、私たちは256×256の解像度の画像で訓練とテストを行い、この同じ解像度でトルコ人に結果を提示しました。

- “FCN-score”

近年の研究[51,54,61,41]は、生成された刺激の識別可能性を擬似距離として測定するために、事前に訓練された意味的分類子を用いて試みた。 直感は、生成された画像が現実的である場合、実際の画像で訓練された分類器は、合成された画像を正しく分類することができるということである。 この目的のために、セマンティックセグメンテーションのための一般的なFCN-8 [38]アーキテクチャを採用し、それをcityscapesデータセットで訓練する。 次に、これらの写真を合成したラベルに対して、合成された写真に分類精度でスコアを付けます。

### 4.2. Analysis of the objective function

方程式の目的のどの構成要素。 4は重要ですか？我々は、L1項、GAN項の効果を分離し、無条件弁別器（GAN、式2）を用いて入力（cGAN、式1）を条件とする弁別器を用いて比較するためにアブレーション試験を行う。図4は、2つのラベル→写真の問題に対するこれらのバリエーションの定性的効果を示しています。 L1だけでは妥当であるが曖昧な結果につながる。 cGANだけでも（式4のλ= 0に設定すると）はるかにシャープな結果が得られますが、特定のアプリケーションで視覚的なアーチファクトが発生します。両方の項を一緒に加えると（λ= 100）これらのアーチファクトが減少します。

都市景観ラベル→写真タスク（表1）のFCNスコアを使用して、これらの観測値を定量化する：GANベースの目的はより高いスコアを達成し、合成された画像がより認識可能な構造を含むことを示す。また、弁別器（GANとラベル付け）からコンディショニングを取り除く効果をテストします。この場合、損失は入力と出力の不一致に不利益を与えません。それは出力が現実的に見えることだけを気にします。この変種はパフォーマンスが非常に悪くなります。

結果を調べると、ジェネレータは入力写真に関係なくほぼ完全に同じ出力を生成するように崩壊したことがわかります。明らかに、この場合、損失は入力と出力の一致の品質を測定することが重要であり、確かにcGANはGANよりも優れた性能を発揮します。ただし、L1項を追加すると、出力が入力を尊重することも奨励されます.L1の損失は、入力と正しく一致するグランド・トゥルース出力と、そうでない可能性のある合成出力との距離に不利益を与えるためです。これに対応して、L1 + GANは、入力ラベルマップを尊重する現実的なレンダリングを作成するのにも有効です。すべての用語L1 + cGANを組み合わせると、同様にうまく機能します。

- Colorfulness

条件付きGANの顕著な効果は、入力ラベルマップに存在しない場合でも空間構造を幻覚させる鮮明な画像を生成することです。 cGANは、スペクトル次元における「鮮鋭化」に類似の効果を有する、すなわち画像をよりカラフルにすると想像することができる。 L1がエッジを正確に見つける場所が不明な場合、L1がぼかしを刺激するのと同様に、画素が取るべきいくつかの妥当なカラー値のどれが不確かな場合には、平均的な灰色がかった色を強調する。特に、L1は、可能な色に対して条件付き確率密度関数の中央値を選択することによって最小化される。一方、敵対的な損失は、原則的に、灰色の出力が非現実的であることを認識し、真の色分布を一致させることを奨励する[23]。図7では、cGANが実際にCityscapesデータセットでこの効果を達成しているかどうかを調べます。プロットはLab色空間の出力カラー値に対する周辺分布を示しています。地上の真理分布は点線で示されている。 L1が地上の真実よりも狭い分布を示し、L1が平均的で灰色がかった色を奨励するという仮説を確認することは明らかである。一方、cGANを使用すると、出力分布が地上真理に近づきます。

### 4.3. Analysis of the generator architecture

U-Netアーキテクチャにより、低レベルの情報をネットワーク全体にショートカットさせることができます。 これはより良い結果につながりますか？ 図5と表2は、U-Netと都市景観のエンコーダデコーダを比較しています。 エンコーダ/デコーダは、UNet内のスキップ接続を切断するだけで作成されます。 エンコーダ/デコーダは、我々の実験で現実的な画像を生成することを学ぶことができない。 U-Netの利点は、条件付きGANに特有のものではないように見えます.U-Netとエンコーダデコーダの両方がL1損失で訓練された場合、U-Netは再び優れた結果を達成します。

### 4.4. From PixelGANs to PatchGANs to ImageGANs

私たちは、1x1 "PixelGAN"から286x286 "ImageGAN" 1までの識別器受容フィールドのパッチサイズNを変化させる効果をテストします。図6はこの分析の定性的結果を示し、表3はFCNスコアを用いた効果を定量化する。このホワイトペーパーでは、指定しない限り、すべての実験で70×70パッチGANを使用します。このセクションでは、すべての実験でL1 + cGAN損失が使用されます。 PixelGANは空間的なシャープネスには影響しませんが、結果のカラフルさを向上させます（図7で数値化）。たとえば、図6のバスは、ネットがL1損失で訓練されている場合はグレーで塗りつぶされていますが、PixelGAN損失では赤色になります。カラーヒストグラムマッチングは画像処理の共通の問題であり[48]、PixelGANは有望な軽量ソリューションとなる可能性がある。16×16 PatchGANを使用すれば、シャープな出力を促進し、良好なFCNスコアを達成するだけでなくタイルアーチファクトにつながる。 70×70 PatchGANは、これらのアーチファクトを軽減し、やや良い類似スコアを達成する。これを超えて、286×286のImageGANまでは、結果の視覚的品質を向上させるようには見えず、実際にはFCNスコアがかなり低くなっています（表3）。 ImageGANには、70×70のPatchGANよりも多くのパラメータと深度があり、訓練するのが難しいためです。

- Fully-convolutional translation

PatchGANの利点は、固定サイズのパッチディスクリミネータを任意に大きな画像に適用できることです。 我々はまた、訓練されたものよりも大きな画像上に、畳み込み的にジェネレータを適用することもできる。 私たちはmap匿名写真タスクでこれをテストします。 256×256画像のジェネレータを訓練した後、512×512画像でテストします。 図8の結果は、このアプローチの有効性を示しています。

### 4.5. Perceptual validation

我々は、地図緯度写真およびグレースケール→色のタスクに関する結果の知覚的リアリズムを検証する。我々の方法で生成された空中写真は、実験結果の18.9％で、L1ベースラインを大幅に上回り、曖昧な結果をもたらし、参加者をだますことはほとんどありませんでした。対照的に、写真→マップ方向では、我々の方法は参加者を6.1％の試行で欺いただけであり、これはL1ベースラインの性能（ブートストラップ試験に基づく）と有意に異ならなかった。これは、剛性のあるジオメトリを持つマップでは、混沌とした航空写真よりも小さな構造的な誤差がより目に見えるためです。我々はImageNet [50]上で色付けを訓練し、[61,34]によって導入されたテスト分割についてテストした。我々の方法は、L1 + cGAN消失を伴って、参加者を22.5％の試行で欺いた（表5）。我々はまた、[61]の結果と、L2喪失を使用した方法の変形をテストした（詳細は[61]を参照）。条件付きGANは[61]のL2変形（ブートストラップテストでは有意でない差異）と同様に評価されましたが、我々の実験では27.8％の参加者をだましていました。私たちは、そのメソッドが、色付けをうまく行うように特別に設計されていることに注意してください。

### 4.6. Semantic segmentation

条件付きGANは、画像処理およびグラフィックスタスクで一般的であるように、出力が非常に詳細または写真である問題に対して有効であるように見える。セマンティックセグメンテーションのように、出力が入力よりも複雑ではない視覚の問題はどうですか？これをテストするために、都市風景写真→ラベル上でcGAN（L1損失あり/なし）を訓練します。図10は定性的結果を示し、定量的な分類精度は表6に報告されています。興味深いことに、L1損失なしで訓練されたcGANは、この問題を合理的な精度で解決することができます。私たちの知る限りでは、これはGANが「ラベル」を生成することを初めて実証したものです。 cGANはいくつかの成功を収めていますが、この問題を解決するために利用可能な最良の方法とはまったく異なります.LG回帰を使用するだけで、表6に示すように、cGANを使用するよりもスコアが高くなります。ビジョンの問題では、グラウンドトゥルーに近い）は、グラフィックタスクよりも曖昧ではなく、L1のような再構成損失で十分です。

### 4.7. Community-driven Research

最初のリリースのペーパーとpix2pixコードベース以来、コンピュータビジョンやグラフィックス実務家、ビジュアルアーティストを含むTwitterコミュニティは、スコープをはるかに超えたさまざまな斬新なイメージ - イメージ変換タスクに我々のフレームワークをうまく適用しました 元の紙の 図11は、＃pix2pixハッシュタグからのほんのいくつかの例を示しています。バックグラウンド除去、パレット生成、スケッチ→ポートレート、スケッチ→ポケモン、「Do Do I Do」ポーズ転送、そして人気のある＃edges2catsおよび#fotogeneratorを含みます。 これらのアプリケーションは創造的なプロジェクトであり、管理された科学的条件では入手できず、リリースしたpix2pixコードの一部の変更に依存する可能性があることに注意してください。 それにもかかわらず、彼らは、イメージからイメージへの翻訳問題の一般的なコモディティツールとして我々のアプローチの約束を示しています。

## 5. Conclusion

この論文の結果は、条件付き対立ネットワークが、特に高度に構造化されたグラフィック出力を伴う多くの画像 - 画像変換タスクにとって有望なアプローチであることを示唆している。 これらのネットワークは、手元のタスクやデータに適応した損失を学習し、さまざまな設定に適用できます。

- Acknowledgments

Richard Zhang氏、Deepak Pathak氏、Shubham Tulsiani氏には有益な議論が寄せられ、Saining Xie氏はHEDエッジ検出器の助けを借りて助けになりました。 この作品は、NSFのSMA-1514512、NGAによって部分的に支持されました。この作品はChristopher Hesse、Kaihu Chen、Jack Qiao、Mario Klingemann、Brannon Dorsey、Gerda Bosman、Ivy Tsai、Yann LeCun 空軍研究所、Intel Corp、Berkeley Deep Drive、Nvidiaによるハードウェア寄付によるNURI、IARPA。 J.Y.Z. Facebook Graduate Fellowshipがサポートしています。 免責事項：ここに含まれる見解と結論は著者のものであり、IARPA、AFRLまたは米国政府の公式の方針または裏書（必ずしも明示または黙示）を表すものであると解釈されるべきではありません。